In [ ]:
import json
import os

import matplotlib.pyplot as plt
import numpy as np

In [ ]:
do_sim_run = False

In [ ]:
graph_dict = json.load(open("graph_dict.json"))

In [ ]:
from simulator import Simulator

In [ ]:
sim = Simulator(graph_dict=graph_dict)

In [ ]:
if do_sim_run:
    sim.run_conf()

In [ ]:
data=np.load("./data/simulation_1.npz")
time_series = data['t']
state_var = data['y']

In [ ]:
# Plot aggregated states

aggregated = np.sum(state_var, axis=2)

fig, ax = plt.subplots()
ax.plot(time_series, aggregated[:, sim.c_idx["S"]], "g")
ax.plot(time_series, aggregated[:, sim.c_idx["E"]], "r")
ax.plot(time_series, aggregated[:, sim.c_idx["I"]], "b")
ax.plot(time_series, aggregated[:, sim.c_idx["R"]], "purple")
ax.legend(["Susceptibles", "Exposed", "Infected", "Recovered"], loc="best") 
plt.show()

In [ ]:
# Plot susceptibles by age

fig, ax = plt.subplots()
ax.plot(time_series, state_var[:, sim.c_idx["S"], 0], "g")
ax.plot(time_series, state_var[:, sim.c_idx["S"], 1], "r")
ax.plot(time_series, state_var[:, sim.c_idx["S"], 2], "b")
ax.legend(["Age group 0-14", "Age group 15-59", "Age group 60+"], loc="best") 
plt.show()

In [ ]:
to_plot = []
peak_sizes = []

daily_data = []

for sim_cnt in range(104):
    # Load data
    data=np.load("./data/simulation_" + str(sim_cnt) + ".npz")
    time_series = data['t']
    state_var = data['y']
    # Save data for multiple time series plot
    aggregated = np.sum(state_var, axis=2)
    to_plot.append((time_series, aggregated[:, sim.c_idx["I"]]))
    
    # Generate daily data
    day = 1
    day_i = []
    days = []
    t_prev = None
    i_prev = None
    for t, y in zip(time_series, aggregated[:, sim.c_idx["I"]]):
        if t_prev is not None:
            if (t - t_prev) > 1.0:
                for d in range(day, int(t) + 1):
                    days.append(d)
                    day_i.append(i_prev)
                day = int(t) + 1
            elif t_prev <= day < t:
                days.append(day)
                day = int(t) + 1
                day_i.append(i_prev)
        else:
            day_i.append(y)
            days.append(t)
        t_prev = t
        i_prev = y
    daily_data.append((days, day_i))
    
    # Save data fpr confidence plot
    i_agg_max = np.amax(aggregated[:, sim.c_idx["I"]])
    peak_sizes.append(i_agg_max)

s = np.std(np.array(peak_sizes))

In [ ]:
# Plot multiple loaded time series

fig, ax = plt.subplots()
for ts, sv in to_plot:
    ax.plot(ts, sv)
plt.show()

In [ ]:
os.makedirs('./plots', exist_ok=True)
fig.savefig('./plots/conf_plot.pdf')

In [ ]:
# Plot multiple loaded time series

fig, ax = plt.subplots()
for ts, sv in daily_data:
    ax.plot(ts, sv)
plt.show()

In [ ]:
# Plot confidence band

fig, ax = plt.subplots()
y_data = np.mean(np.array(daily_data[:][1]), axis=0)
ax.fill_between(daily_data[0][0], y_data - 2 * s, y_data + 2 * s, alpha=0.3)
ax.plot(daily_data[0][0], y_data, "r")
plt.show()

In [ ]:
os.makedirs('./plots', exist_ok=True)
fig.savefig('./plots/conf_plot_2.pdf')

In [ ]:
print(np.array([len(v) for _, v in daily_data]))
print(np.array([len(t) for t, _ in daily_data]))